In [ ]:
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import os
import time

from skimage import morphology
from skimage.segmentation import expand_labels, watershed
from scipy.ndimage import convolve1d
from dask_ml.cluster import KMeans
from dask_ml.decomposition import PCA

from dask_image import imread
from dask_image import ndmeasure
from dask_image import ndfilters
from dask import array as da
from dask.distributed import Client, progress

from eigenvalues_from_unique import eigenvalues_from_unique

import napari

In [ ]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=1, memory_limit='12GB')
client

In [ ]:
#Specific is the name of the folder of the orginal data
specific = 'Whole Hand Test S232312 Left'
#which_finger should be 1,2,3, or 4, corresponding to the index, middle, ring, and pinky fingers
which_finger = 2

In [ ]:
finger_dir = {1: '../fingers/index/full/' + specific + '/',
              2: '../fingers/middle/full/' + specific + '/',
              3: '../fingers/ring/full/' + specific + '/',
              4: '../fingers/pinky/full/' + specific + '/',
             }
temp_eigs_dir = {1: '../fingers/index/full/' + specific + ' - eigens/',
              2: '../fingers/middle/full/' + specific + ' - eigens/',
              3: '../fingers/ring/full/' + specific + ' - eigens/',
              4: '../fingers/pinky/full/' + specific + ' - eigens/',
             }
temp_eigs_dir_2 = {1: '../fingers/index/full/' + specific + ' - eigens_2/',
              2: '../fingers/middle/full/' + specific + ' - eigens_2/',
              3: '../fingers/ring/full/' + specific + ' - eigens_2/',
              4: '../fingers/pinky/full/' + specific + ' - eigens_2/',
             }

In [ ]:
#If the file is small enough for your computer, 
#then add ".compute" to the end of the line below below.
finger = da.from_npy_stack(finger_dir[which_finger])
spacing = np.load('../spacing/' + specific + '.npy')

In [ ]:
#DD = da.from_npy_stack('temp-derivatives/')

In [ ]:
#n = 10**6
#d1, d2, d3, _ = DD.shape
#DD_list = DD.reshape((d1*d2*d3, 6))
#index = da.random.choice(DD_list.shape[0], n) 
#DD_samp = DD_list[index]
#DD_samp

In [ ]:
#Force Dask to compute the sample now in order to save time later
#da.to_npy_stack('sample-derivatives/', DD_samp)

In [ ]:
#Reload the data from the temporary file
DD_samp = da.from_npy_stack('sample-derivatives/')

In [ ]:
DD_samp

In [ ]:
def viete(the_six):
    if len(the_six) == 6:
        #the_six should be formatted as the [xx, xy, yy, yz, zz, zx] derivatives
        # of the Gaussian convolved with the image
        a11, a12, a22, a23, a33, a13 = the_six
        #The characteristic polynomial of the 3x3 matrix will have coefficients:
        trace_av = np.average([a11, a22, a33])
        P_shifted = np.array([[a11-trace_av,a12,a13],
                      [a12,a22-trace_av,a23],
                      [a13,a23,a33-trace_av]])
        _,_,p,q = np.poly(P_shifted)
        neg_p_sqrt = np.sqrt(-p)
        sqrt_3 = np.sqrt(3)
        if p >= -10**(-5): #This will only happen if all off-diagonals are basically zero and
            # all diagonals are basically identical.
            eigens = np.array([trace_av]*3)
        elif 27*q**2 > -4*p**3 - 10**(-5):
            #This can only happen when the discriminant is basically zero
            # which will mean cos(theta) = sign(q).  In either case, cosine of one of
            # the trisected angels will also be sign(q).
            la1 = neg_p_sqrt * 2 * np.sign(q) / sqrt_3 - trace_av
            la23 = neg_p_sqrt * (-np.sign(q)) / sqrt_3 - trace_av
            eigens = np.array(sorted([la1, la23, la23], reverse=True))
        else:
            theta = np.arccos(3*sqrt_3*q/(2*neg_p_sqrt**3))
            cs = np.cos(theta/3)
            sn = np.sqrt(1 - cs**2)
            la1 = neg_p_sqrt * 2 * cs / sqrt_3 - trace_av
            la2 = neg_p_sqrt * (-cs + sqrt_3*sn) / sqrt_3 - trace_av
            la3 = neg_p_sqrt * (-cs - sqrt_3*sn) / sqrt_3 - trace_av
            eigens = np.array(sorted([la1, la2, la3], reverse=True))
        return eigens
    else:
        print(f'Viete: The chosen axis must be of length 6, but instead, got: {the_six} which is length {len(the_six)}')
        return np.zeros(3)

In [ ]:
eigs = da.apply_along_axis(viete, 1, DD_samp)

In [ ]:
%timeit -r 10 -n 10 da.to_npy_stack(temp_eigs_dir[which_finger], eigs)

In [ ]:
eigs_2 = da.apply_along_axis(eigenvalues_from_unique, 1, DD_samp)

In [ ]:
%timeit -r 10 -n 10 da.to_npy_stack(temp_eigs_dir_2[which_finger], eigs_2)

In [ ]:
eigs = da.from_npy_stack(temp_eigs_dir[which_finger])

In [ ]:
eigs